In [8]:
%%python
from coppertop import *
"/Users/david/shared/repos/github/DangerMouseB/bones/python" >> EnsurePath
"bones" >> Unload
from bones.coppertop import *
from bones.combinators import MatchAtomicTag, MatchTagClass, MatchTagInstance, parseUsing, getAst
from bones.basic_parsers import *
from bones.interpreter import BonesResult
from bones.pragmatics.symbols import SymRegister

### Combinator cheat sheet
```
a >> f - if matches a return f(a) i.e. postProcess
a // b - list of a seperated by b
~a     - zero or one a
+a     - zero or more a
a + b  - a then b
a | b  - a or b
```

In [9]:
%%python
catcher = l_paren + integer + r_paren
'(314)' >> parseUsing(..., catcher, None, SymRegister()) >> PP
'(314' >> parseUsing(..., catcher, None, SymRegister()) >> PP
'314)' >> parseUsing(..., catcher, None, SymRegister())

(({L_PAREN}, 314), {R_PAREN}) --- <( <l_paren> then <integer> then <r_paren> )>
NoMatch(matchedSoFar=({L_PAREN}, 314) --- <( <l_paren> then <integer> )>)
NoMatch(matchedSoFar=None)


In [10]:
%%python

catcher = integer // (plus_binop_src | star_binop_src)   # No precedence
parse = parseUsing(..., catcher, None, SymRegister())
src = '2 + 3 * 4'
expected = ((2, '+', 3), '*', 4)
ast = src >> parse >> PP
ast >> getAst >> AssertEqual >> expected


catcher = integer // star_binop_src // plus_binop_src   # with precedence * over +
parse = parseUsing(..., catcher, None, SymRegister())
src = '2 + 3 * 4'
expected = (2, '+', (3, '*', 4))
ast = src >> parse >> PP
ast >> getAst >> AssertEqual >> expected;

'3 + 4' >> parse >> PP
'3 * 4' >> parse >> PP
'3' >> parse >> PP;

((2, '+', 3), '*', 4) --- < <integer> sep_by <[ <plus_binop_src> or <star_binop_src> ]> >
(2, '+', (3, '*', 4)) --- < < <integer> sep_by <star_binop_src> > sep_by <plus_binop_src> >
(3, '+', 4) --- < < <integer> sep_by <star_binop_src> > sep_by <plus_binop_src> >
(3, '*', 4) --- < < <integer> sep_by <star_binop_src> > sep_by <plus_binop_src> >
3 --- < < <integer> sep_by <star_binop_src> > sep_by <plus_binop_src> >


In [11]:
%%python
parse = parseUsing(..., +integer, None, SymRegister())
'' >> parse >> getAst >> AssertEqual >> []
'1' >> parse >> getAst >> AssertEqual >> [1]
'1 2' >> parse >> getAst >> AssertEqual >> [1, 2]

parse = parseUsing(..., ~integer, None, SymRegister())
'' >> parse

Absent:<integer> --- <opt:<integer>>


### Adding the handler to the kernel 

In [12]:
%%python
def execute(src, kernel):
    # hello {fred, joe, sally}
    src = src.strip()
    if src[0:1] == '"':
        # handle "David" :me
        raise NotImplementedError("literal strings not handled yet")
    elif src[0:1] == 'h':
        try:
            assert src[0:6] == "hello ", "not hello"
            assert src[6:].strip() in ['fred', 'joe', 'sally'], "not fred, joe nor sally"
            return kernel.OK, [src[6:].strip() + ' says "Hello"']
        except AssertionError as ex:
            return kernel.ERROR, [ex.args[0]]
    return kernel.ERROR, ['Unkown syntax']


In [13]:
%%python
catcher = integer // star_binop_src // plus_binop_src    # with precedence * over +
parse = parseUsing(..., catcher, None, SymRegister())

class BonesHandler(object):
    def parse(self, src, kernel):
        return kernel.OK, [src >> parse]

handler = BonesHandler()
handler.execute = execute
_kernel.handlers['bones'] = handler
_kernel.defaultHandler = 'bones'

### Inspecting the parse result

In [14]:
%%parse
2 + 3 * 4

%%execute
hello sally

[1] %%parse
(2, '+', (3, '*', 4)) --- < < <integer> sep_by <star_binop_src> > sep_by <plus_binop_src> >
[2] %%execute
sally says "Hello"


### Execution

In [15]:
hello 
   freds,
   joes,

not fred, joe nor sally


In [16]:
""

Traceback (most recent call last):
  File "/Users/david/shared/repos/github/DangerMouseB/bones_kernel/bones_kernel/kernel.py", line 258, in do_execute
    outcome, values = handlerFn(arg, self._sharedKernel)
  File "2:15", line 7, in execute
NotImplementedError: literal strings not handled yet


In [17]:
ugh

Unkown syntax


In [18]:
hello sally

sally says "Hello"


In [19]:
fred

Unkown syntax


In [20]:
1+1

Unkown syntax
